In [1]:
import timeit
import importlib
import relevancer as rlv
import pandas as pd
from sklearn.naive_bayes import MultinomialNB 
from bson.objectid import ObjectId
from collections import Counter
import numpy as np

pd.set_option("display.max_colwidth",200)


In [2]:
# this is just to load the latest version of the Relevancer after we edit the code.
importlib.reload(rlv)
active_col = "active_text"
rlv.set_active_column(active_col)


In [3]:
#importlib.reload(rlv)
my_token_pattern=r"[#@]?\w+\b|[\U00010000-\U0010ffff]"
rlvdb, rlvcl = rlv.connect_mongodb(configfile='elifdb.ini',coll_name="testcl")
# set active columns
active_col = "active_text"
rlv.set_active_column(active_col)

In [4]:
importlib.reload(rlv)

# after the first iteration, the annotated clusters should be excluded from the clustering.
# read this file from the tagged collection.
annotated_tw_ids = ['563657483395530753', '563662532326330370', '563654330041909248', '563654944927281152', '563657924233289728', '563661021559390208', '563651950386757632', '563657164317667328', '563660271810383872', '563662538949160960'] #You should get the actual annotated tweet ids from the annotated tweets collection.
#annotated_tw_ids = []
# mongo_query=({'_id': {'$gte': begin, '$lte': end},'lang':'en'})
tweetlist = rlv.read_json_tweet_fields_database(rlvcl, mongo_query=({}), read_fields={'text': 1, 'id_str': 1, '_id': 0, 'user_id': 1}, tweet_count=-1, annotated_ids=annotated_tw_ids)#=tweetsDF)

rlv.logging.info("Number of tweets:" + str(len(tweetlist)))
print("Number of tweets:",len(tweetlist))

Number of tweets: 391649


In [5]:
tweetsDF = rlv.create_dataframe(tweetlist)
tweetsDF.head()

391649


,id_str,is_retweet,text,user_id
0,615329612881612800,False,RT @kararsizatom: Do you have an idea to stop this genocide? #StopTerrorisminChina @carlbildt http://t.co/edq1P8nTtw,1381676425
1,605474184714543104,False,RT @RumBegum: This situation won't go viral because it's Muslims being killed not Muslims killing #Rohingya #UNMustSaveBurmaMuslims http://…,2831033527
2,613073957319782400,False,"@eurireland The second GENOCIDE against #Biafrans as promised by #Buhari has begun, 3 days of unreported aerial Bombardment in #Biafraland",2188829916
3,613337519753555968,False,RT @WachukwuGodwin: @BBCScotlandNews #Buhari &amp; Nigeria declare war on #Biafra killing 100s of innocent Biafrans wit Napalm bombs GENOCIDE h…,1173943710
4,605477505651359744,False,#ModiMinistry Obama says Myanmar needs to end discrimination of Rohingya to succeed\n| Reuters http://t.co/zmagjFtMH8,2425268995


In [6]:
len(tweetsDF)

391649

In [187]:
#tweetsDF.to_pickle("20151005_tweetsDF_Genocide")

In [8]:
# Be careful do not overwrite tweetsDFBackUP with a modified tweetsDF
#tweetsDFBackUP = tweetsDF.copy()
#len(tweetsDFBackUP)

In [14]:
# Get a clean copy 
#importlib.reload(rlv)
tweetsDF = tweetsDFBackUP.copy()
len(tweetsDF)

NameError: name 'tweetsDFBackUP' is not defined

In [7]:
tweetsDF[active_col] = tweetsDF["text"].copy()
tweetsDF = rlv.tok_results(tweetsDF, elimrt = True)


Available attributes of the tweets: Index(['id_str', 'is_retweet', 'text', 'user_id', 'active_text', 'texttok'], dtype='object')
<class 'pandas.core.frame.DataFrame'>
Int64Index: 391649 entries, 0 to 391648
Data columns (total 6 columns):
id_str         391649 non-null object
is_retweet     391649 non-null bool
text           391649 non-null object
user_id        391649 non-null object
active_text    391649 non-null object
texttok        391649 non-null object
dtypes: bool(1), object(5)
memory usage: 18.3+ MB

tweet set summary: None
0                                RT @kararsizatom: Do you have an idea to stop this genocide? #StopTerrorisminChina @carlbildt http://t.co/edq1P8nTtw
1        RT @RumBegum: This situation won't go viral because it's Muslims being killed not Muslims killing #Rohingya #UNMustSaveBurmaMuslims http://…
2          @eurireland The second GENOCIDE against #Biafrans as promised by #Buhari has begun, 3 days of unreported aerial Bombardment in #Biafraland
3    RT @

/home/narkem/workspaces/wsPython/Relevancer/relevancer.py:508: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  tweetsDF[active_column] = tweetsDF[tok_result_lower_col].copy()


In [16]:
len(tweetsDF)

175524

In [8]:
tweetsDF = rlv.normalize_text(tweetsDF)
len(tweetsDF), tweetsDF.columns

(175524,
 Index(['id_str', 'is_retweet', 'text', 'user_id', 'active_text', 'texttok',
        'is_notrt'],
       dtype='object'))

In [190]:
#importlib.reload(rlv)
#rlv.set_active_column(active_col)

In [11]:
def eliminate_duplicates_recursively(df, duplicate_elim_func):
            
        print("starting, length:",len(df))
        df = df.reindex(np.random.permutation(df.index))
        df.reset_index(inplace=True, drop=True)
        
        tmp_df2 = pd.DataFrame()
        for i in range(0, len(df), 10000):
            tmp_unique = duplicate_elim_func(df[i:i+1000], similarity_threshold=0.20, debug=True, debug_threshold=10000)
            tmp_df2 = pd.concat([tmp_df2, tmp_unique], ignore_index=True)
            
        if len(df) > len(tmp_df2):
            print(str(len(df) - len(tmp_df2))+" tweets were eliminated!")
            return eliminate_duplicates_recursively(tmp_df2, duplicate_elim_func)
        
        return df

In [12]:
tweetsDF_uniq = eliminate_duplicates_recursively(tweetsDF.copy(), rlv.get_and_eliminate_near_duplicate_tweets)


starting, length: 175524
<class 'pandas.core.frame.DataFrame'>
Int64Index: 779 entries, 0 to 585
Data columns (total 7 columns):
id_str         779 non-null object
is_retweet     779 non-null bool
text           779 non-null object
user_id        779 non-null object
active_text    779 non-null object
texttok        779 non-null object
is_notrt       779 non-null bool
dtypes: bool(2), object(5)
memory usage: 38.0+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 788 entries, 10000 to 10348
Data columns (total 7 columns):
id_str         788 non-null object
is_retweet     788 non-null bool
text           788 non-null object
user_id        788 non-null object
active_text    788 non-null object
texttok        788 non-null object
is_notrt       788 non-null bool
dtypes: bool(2), object(5)
memory usage: 38.5+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 805 entries, 20001 to 20367
Data columns (total 7 columns):
id_str         805 non-null object
is_retweet     805 non-null bool
t

In [13]:
len(tweetsDF_uniq)

975

In [14]:
tweetsDF_uniq.to_pickle("20151005_unique_genocide_tweets.pickle")

In [15]:
tweetsDF_uniq[["active_text"]][:10]

,active_text
0,one year of we charge genocide | we charge genocide urlurlurl #chicopwatch - an incredible year of work.
1,usrusrusr the cause of terrorism carried out by the dictator and his henchmen murder and genocide examples of presidents in the middle east p
2,"the supporter of pol pot and red khmer's genocide, mr brzeziński, is pointing to wisdom about putin. seems legit. urlurlurl"
3,"abandoning the victims of genocide in darfur urlurlurl #gameinsight, #android, #androidgames, #ipad, #ipadgames, #openfollow…"
4,all i want for christmas is for that mosquito genocide program to come to dc.
5,usrusrusr urlurlurl loonwatch compares a government with the power to do so with(genocide) with usrusrusr
6,usrusrusr &amp; usrusrusr just admit ur helplessness 2 stop #hazaras killings in quetta n they will stop blaming u 4 their genocide!
7,pl4et: #genocide of #chinese #communist party in #eastturkistan .( #uyghur ) \nurlurlurl\n#china #ccp #freeuyghur #humanrights…
8,"it gets better!!! ""are you a spy? a terrorist? have you ordered an attack or genocide in the us? killed someone? recruited child soldiers?"""
9,government to provide religious education to rohingya refugees urlurlurl


In [16]:
importlib.reload(rlv)
rlv.set_active_column(active_col)

In [17]:
cluster_list = rlv.create_clusters(tweetsDF_uniq, my_token_pattern, min_dist_thres=0.725, max_dist_thres=0.875, min_max_diff_thres=0.4, nameprefix='1-', min_clusters=100, user_identifier='user_id')

In [18]:
print("Number of clusters:",len(cluster_list))
print("available cluster information:", cluster_list[0].keys())

Number of clusters: 123
available cluster information: dict_keys(['user_entropy', 'twids', 'cno', 'cstr', 'cnoprefix', 'ctweettuplelist', 'rif'])


In [19]:
i = 0

In [20]:
#i = i-2
print("No:",cluster_list[i]['cno'])
print("CStr:",cluster_list[i]['cstr'])
print("Cluster Tuple List:")
print(*[(c[0],c[2]) for c in cluster_list[i]['ctweettuplelist']], sep='\n')
# Add any field you want to observe.
i+=1

No: 11
CStr: cluster number and size are: 11    12

Cluster Tuple List:
(0.70357675493996663, 'usrusrusr they also started the armenian genocide.')
(0.72593853185451451, 'usrusrusr 11 years before the armenian genocide began')
(0.74056366460613288, 'denial of the armenian genocide - right now urlurlurl')
(0.7730663724399286, 'a new book on the armenian genocide | mark movsesian | first things urlurlurl')
(0.77737895573242521, 'exhibition at center for studies of holocaust and religious minorities: the armenian genocide  urlurlurl')
(0.7928147381693027, 'pope: armenian ww1 deaths genocide')
(0.81474133725487174, "people who deny the armenian genocide when it's 2015 urlurlurl")
(0.83135878724273748, 'majlis mp: tendencies to recognize armenian genocide may be observed in iran soon: tendencies to recognize the... urlurlurl')
(0.83416216284619338, 'usrusrusr it\'s called "politics." he also hasn\'t recognized the armenian genocide for the same reason.')
(0.83727510021287699, 'rostomzavaria

In [21]:
print("Reverse Frequency of the terms:")
print(i)
for k, v in Counter({int(k):v for k,v in cluster_list[i-1]['rif'].items()}).items():
    if k>1:
        print(k,":",v)

Reverse Frequency of the terms:
1
2 : ['then you', 'tendencies to', 'of the', 'it s', '11', 'to recognize', 'recognize', 'when', 'amp', 'i', 'for', 'tendencies', 's', 'also', 'and', 'then', 'it']
3 : ['you', 'of', 'to']
4 : ['usrusrusr']
6 : ['urlurlurl']
10 : ['the armenian']
11 : ['armenian genocide']
12 : ['armenian', 'genocide']
14 : ['the']


In [22]:
rlvdb2, rlvcl2 = rlv.connect_mongodb(configfile='ebasar2.ini',coll_name="testcl")

collection_name = 'genocide_clusters_20151005'
rlvdb2[collection_name].insert(cluster_list) #Each iteration results with a candidate cluster list. Each iteration will have its own list. Therefore they are not mixed.
print("Clusters were written to the collection:", collection_name)

Clusters were written to the collection: genocide_clusters_20151005


In [241]:
import pickle
with open("genocide_clusters_20151005.pickle", 'wb') as f:
    pickle.dump(cluster_list, f, pickle.HIGHEST_PROTOCOL)
    print("wrote km user to pickle")
        
# if read_km_user:
#     with open(fname, 'rb') as f:
#         km_user = pickle.load(f)
#         print("read km_user from pickle")

wrote km user to pickle


In [248]:
cluster_list[0].keys()

dict_keys(['ctweettuplelist', 'twids', 'cnoprefix', '_id', 'rif', 'cno', 'cstr', 'user_entropy'])

In [249]:
cluster_list_small = []
for c in cluster_list:
    cluster_list_small.append({k:v for k,v in c.items() if k in ["cno","cstr","user_entropy","rif","_id","cnoprefix",'twids']})

In [250]:


collection_name = 'genocide_clusters_small_20151005'
rlvdb[collection_name].insert(cluster_list_small) #Each iteration results with a candidate cluster list. Each iteration will have its own list. Therefore they are not mixed.
print("Clusters were written to the collection:", collection_name)

OperationFailure: quota exceeded